In [2]:
import pandas as pd
from nilearn.plotting import plot_design_matrix
from nilearn.image import load_img
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.spatial import distance
from scipy.io import loadmat
import sys
sys.path.append('/home/ubuntu/repos/learning-habits-analysis')
from utils.data import Subject
from utils.data import get_betamap_paths, load_participant_list

In [3]:
def load_data(first_lvl_dir, contrast):
    contrast_dir = os.path.join(first_lvl_dir, contrast)
    nii_files = glob.glob(os.path.join(contrast_dir, "*.nii"))
    # sort to have consistent order
    nii_files = sorted(nii_files)
    imgs = [load_img(f) for f in nii_files]
    data = np.stack([img.get_fdata() for img in imgs], axis=-1)
    return data, nii_files

# Loading masks

In [4]:
mask_dir = '/mnt/data/learning-habits/masks/MNI152NLin2009cAsym'
mask_files = [os.path.join(mask_dir, f) for f in os.listdir(mask_dir)]

In [5]:
mask_files

['/mnt/data/learning-habits/masks/MNI152NLin2009cAsym/parietal_AAL_MNI152NLin2009cAsym.nii',
 '/mnt/data/learning-habits/masks/MNI152NLin2009cAsym/habit_Guida2022_MNI152NLin2009cAsym.nii',
 '/mnt/data/learning-habits/masks/MNI152NLin2009cAsym/striatum_bartra2013_MNI152NLin2009cAsym.nii',
 '/mnt/data/learning-habits/masks/MNI152NLin2009cAsym/vmpfc_bartra2013_MNI152NLin2009cAsym.nii',
 '/mnt/data/learning-habits/masks/MNI152NLin2009cAsym/motor_HMAT_MNI152NLin2009cAsym.nii']

In [17]:
# Load masks as nilearn Nifti1Masker objects for ROI extraction
from nilearn.maskers import NiftiMasker
masks = {}
for mask_file in mask_files:
    roi_name = (os.path.basename(mask_file)).split('_')[0]
    mask_img = load_img(mask_file)
    masker = NiftiMasker(mask_img=mask_img)
    masks[roi_name] = masker

In [18]:
masks.pop('parietal');
masks.pop('motor');

# GLM2 - all runs combined

In [8]:
#first_lvl_dir = "/home/ubuntu/data/learning-habits/spm_format_noSDC/outputs/glm2_all_runs_scrubbed_2025-12-11-12-44"
first_lvl_dir = "/home/ubuntu/data/learning-habits/spm_outputs_noSDC//glm2_all_runs_scrubbed_2025-12-11-12-44"

In [9]:
os.listdir(first_lvl_dir)

['contrast-05_second_stimxqval',
 'contrast-01_first_stim',
 'contrast-12_hval_diff',
 'second-lvl',
 'contrast-08_purple_frame',
 'contrast_list_order_allruns.txt',
 'contrast-09_qval_sum',
 'contrast-10_hval_sum',
 'contrast-11_qval_diff',
 'contrast-06_second_stimxhval',
 'contrasts_manifest.tsv',
 'contrast-04_second_stim',
 'contrast-07_response',
 'contrast-02_first_stimxqval',
 'contrast-03_first_stimxhval']

In [12]:
#spm_path = os.path.join(first_lvl_dir,'sub-01','SPM.mat')
#SPM = loadmat(spm_path, squeeze_me=True, struct_as_record=False)['SPM']
#SPM.xX.name

In [13]:
contrasts = ['contrast-02_first_stimxqval',
             'contrast-03_first_stimxhval',
             'contrast-05_second_stimxqval',
             'contrast-06_second_stimxhval']

data = {}
for contrast in contrasts:
    maps, nii_files = load_data(first_lvl_dir, contrast)
    data[contrast] = {'maps': maps, 'nii_files': nii_files}

In [19]:
# Extract mean beta values for each ROI and plot as bar plots
roi_means = {roi: [] for roi in masks.keys()}
subjects = []

# Fit once per mask
for roi, masker in masks.items():
    masker.fit()

roi_means = {roi: [] for roi in masks}
subjects = []

for contrast in contrasts:
    nii_files = data[contrast]["nii_files"]

    # record subjects once per contrast (adjust if you don't want repeats across contrasts)
    subjects.extend([os.path.basename(f) for f in nii_files])

    for roi, masker in masks.items():
        # Batch: (n_imgs, n_voxels_in_roi)
        X = masker.transform(nii_files)

        # Mean within ROI per image -> (n_imgs,)
        roi_means[roi].extend(np.asarray(X).mean(axis=1).tolist())

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f018272dc00>>
Traceback (most recent call last):
  File "/home/ubuntu/miniforge3/envs/neuroim/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [20]:
# Convert to DataFrame for plotting
plot_df = pd.DataFrame(roi_means)
plot_df['subject'] = subjects